# Imports

In [ ]:
import os
import time
from typing import List, Tuple, Dict, Optional, Any

import pandas as pd
import numpy as np
import sqlite3

# Utils

In [ ]:
def get_table_schema(cur, table_name: str):
    """Return the schema of a table
    
   Columns in the result set include:
       "name" (its name);
       "type" (data type if given, else '');
       "notnull" (whether or not the column can be NULL);
       "dflt_value" (the default value for the column); and
       "pk" (either zero for columns that are not part of the primary key, or the 1-based index of the column within the primary key).
   """
    res = cur.execute(f"PRAGMA table_info({table_name});")
    return res.fetchall()
    
def print_table_schema(cur, table_name: str, schema: Tuple[str]):
    """Print the schema of a table
      
      Columns in the result set include:
          "name" (its name);
          "type" (data type if given, else '');
          "notnull" (whether or not the column can be NULL);
          "dflt_value" (the default value for the column); and
          "pk" (either zero for columns that are not part of the primary key, or the 1-based index of the column within the primary key).
    """
    print(f'Schema for `{table_name}`')
    for col in schema:
        print(f'{col[1]:<14}, type:{col[2]:<12} notnull:{col[3]}, dflt:{col[4]}, pk:{col[5]}')
    
def query_as_list(cur, query: str, limit: int = 10):
    """Run a query and return the first limit results."""
    res = cur.execute(query)
    res = res.fetchall()
    return res[:min(limit,len(res))]
    
def print_query(cur, query: str, limit: int = 10):
    """Run a query and print the first 10 results."""
    qal = query_as_list(cur, query, limit)
   
    # Print fields.
    num_fields = len(cur.description)
    field_names = [i[0] for i in cur.description]
    print(f'Cols: {field_names}')

    # Print results.
    for i, r in enumerate(qal):
        print(f'{i}: {r}')

def query_as_list_v2(cur, query: str, params: List[Any], limit: int = 10):
    """Run a query and return the first limit results."""
    if len(params):
        res = cur.execute(query, params)
    else:
        res = cur.execute(query)
    res = res.fetchall()
    return res[:min(limit,len(res))]
    
def print_query_v2(cur, query: str, params: List[Any], limit: int = 10):
    """Run a query and print the first 10 results."""
    for i, r in enumerate(query_as_list(cur, query, params, limit)):
        print(f'{i}: {r}') 

# Load Data SQL

## Cleaned Data

In [ ]:
create_menu_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "menu_cleaned.sql")
create_menu_page_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "menu_page_cleaned.sql")
create_menu_item_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "menu_item_cleaned.sql")
create_dish_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "dish_cleaned.sql")
menu_db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "menudb.db")
con = sqlite3.connect(menu_db_path)
cur = con.cursor()

# Load our exported OpenRefine SQL script.
with open(create_menu_path) as f:
    db_menu_create = f.read()
    
with open(create_menu_page_path) as f:
    db_menu_page_create = f.read()
    
with open(create_menu_item_path) as f:
    db_menu_item_create = f.read()
    
with open(create_dish_path) as f:
    db_dish_create = f.read()

# Create the database using our creation script.
res = cur.executescript(db_menu_create)
con.commit()

# Create the database using our creation script.
res = cur.executescript(db_menu_page_create)
con.commit()

# Create the database using our creation script.
res = cur.executescript(db_menu_item_create)
con.commit()

# Create the database using our creation script.
res = cur.executescript(db_dish_create)
con.commit()


def MaybeDropColumnsMenu(con, cur, tbl_name = 'nypl_menu_step_2_sql_export_demo'):
    cols_to_drop = ['name','notes', 'occasion', 'place', 'keywords', 'page_count', 'dish_count',]
    template = lambda col: f'ALTER TABLE {tbl_name} DROP COLUMN {col};'
    templates = [template(col) for col in cols_to_drop]
    for t in templates:
        res = cur.executescript(t)
    con.commit()
    
def MaybeDropColumnsMenuItem(con, cur, tbl_name = 'menuItem'):
    cols_to_drop = ['created_at', 'updated_at', 'xpos', 'ypos']
    template = lambda col: f'ALTER TABLE {tbl_name} DROP COLUMN {col};'
    templates = [template(col) for col in cols_to_drop]
    for t in templates:
        res = cur.executescript(t)
    con.commit()

def MaybeDropColumnsMenuPage(con, cur, tbl_name = 'menuPage'):
    cols_to_drop = ['page_number', 'image_id', 'full_height', 'full_width','uuid']
    template = lambda col: f'ALTER TABLE {tbl_name} DROP COLUMN {col};'
    templates = [template(col) for col in cols_to_drop]
    for t in templates:
        res = cur.executescript(t)
    con.commit()

def MaybeDropColumnsDish(con, cur, tbl_name = 'Dish'):
    return
    # cols_to_drop = []
    # template = lambda col: f'ALTER TABLE {tbl_name} DROP COLUMN {col};'
    # templates = [template(col) for col in cols_to_drop]
    # for t in templates:
    #     res = cur.executescript(t)
    # con.commit()

[f(con, cur) for f in [MaybeDropColumnsMenu, MaybeDropColumnsMenuItem, MaybeDropColumnsMenuPage, MaybeDropColumnsDish]]

In [ ]:
# Row null/blank deletions will go here.

def MaybeDropRowMenu(con, cur, tbl_name: str = 'nypl_menu_step_2_sql_export_demo'):
    script = '''
        DELETE FROM {0}
        WHERE
            sponsor = '' AND
            event = '' AND
            venue = '' AND
            location = ''
    '''
    res = cur.executescript(string.format(script, tbl_name))
    con.commit()
    

# Not needed. Done in csv and updated sql
def MaybeDropRowDish(con, cur, tbl_name: str = 'dish'):
    # First appeared, last_appeared.
    # Menu appeared, times_appeared.
    # Ahmad need to add a SORT BY to final results that ranks the result by the Menu Appeared, and by the First appeared.
    pass

In [ ]:
# We'll have to join all the tables later, let's do this once in a temp table.
def CreateJoinedTable(con, cur):
    q = """
        DROP TABLE IF EXISTS MenuJoined;
        CREATE TEMP TABLE MenuJoined AS
        SELECT
            Menu.id AS MenuId,
            menuPage.id AS MenuPageId,
            menuItem.id AS MenuItemId,
            Dish.id AS DishId,
            *
        FROM
          nypl_menu_step_2_sql_export_demo AS Menu INNER JOIN
          menuPage ON Menu.id = menuPage.id INNER JOIN
          menuItem ON menuPage.id = menuItem.menu_page_id INNER JOIN
          Dish ON Dish.id = menuItem.dish_id
        ;
    """
    res = cur.executescript(q)
    con.commit()
    
CreateJoinedTable(con, cur)

## Uncleaned Data

In [ ]:
create_uncleaned_menu_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "menu_uncleaned.sql")
create_uncleaned_menu_page_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "menu_page_uncleaned.sql")
create_uncleaned_menu_item_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "menu_item_uncleaned.sql")
create_uncleaned_dish_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "dish_uncleaned.sql")
uncleaned_menu_db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', "uncleaned_menudb.db")
ucon = sqlite3.connect(uncleaned_menu_db_path)
ucur = ucon.cursor()

# Load our exported OpenRefine SQL script.
with open(create_uncleaned_menu_path) as f:
    db_menu_create = f.read()
    
with open(create_uncleaned_menu_page_path) as f:
    db_menu_page_create = f.read()
    
with open(create_uncleaned_menu_item_path) as f:
    db_menu_item_create = f.read()
    
with open(create_uncleaned_dish_path) as f:
    db_dish_create = f.read()

# Create the database using our creation script.
ures = ucur.executescript(db_menu_create)
ucon.commit()

# Create the database using our creation script.
ures = ucur.executescript(db_menu_page_create)
ucon.commit()

# Create the database using our creation script.
ures = ucur.executescript(db_menu_item_create)
ucon.commit()

# Create the database using our creation script.
ures = ucur.executescript(db_dish_create)
ucon.commit()


In [ ]:
zipped = zip([MaybeDropColumnsMenu, MaybeDropColumnsMenuItem, MaybeDropColumnsMenuPage, MaybeDropColumnsDish],
             ['menu_uncleaned', 'menu_item_uncleaned', 'menu_page_uncleaned', 'dish_uncleaned'])
[f(ucon, ucur, tbl) for f, tbl in zipped]

In [ ]:
# We'll have to join all the tables later, let's do this once in a temp table.
def CreateJoinedTableUnclean(con, cur):
    q = """
        DROP TABLE IF EXISTS MenuJoined;
        CREATE TEMP TABLE MenuJoined AS
        SELECT
            Menu.id AS MenuId,
            menuPage.id AS MenuPageId,
            menuItem.id AS MenuItemId,
            Dish.id AS DishId,
            *
        FROM
          menu_uncleaned AS Menu INNER JOIN
          menu_page_uncleaned AS menuPage ON Menu.id = menuPage.id INNER JOIN
          menu_item_uncleaned AS menuItem ON menuPage.id = menuItem.menu_page_id INNER JOIN
          dish_uncleaned AS Dish ON Dish.id = menuItem.dish_id
        ;
    """
    res = cur.executescript(q)
    con.commit()
    
CreateJoinedTableUnclean(ucon, ucur)

## List Clean Schema and Examples

In [ ]:
# Fetch some sample results.
print('\n\n')
print_table_schema(cur, 'menuItem', get_table_schema(cur, 'menuItem'))
print_query(cur, "SELECT * FROM menuItem LIMIT 10")

# Fetch some sample results.
print('\n\n')
print_table_schema(cur, 'nypl_menu_step_2_sql_export_demo', get_table_schema(cur, 'nypl_menu_step_2_sql_export_demo'))
print_query(cur, "SELECT * FROM nypl_menu_step_2_sql_export_demo LIMIT 10")
    
# Fetch some sample results.
print('\n\n')
print_table_schema(cur, 'menuPage', get_table_schema(cur, 'menuPage'))
print_query(cur, "SELECT * FROM menuPage LIMIT 10")

# Fetch some sample results.
print('\n\n')
print_table_schema(cur, 'Dish', get_table_schema(cur, 'Dish'))
print_query(cur, "SELECT * FROM Dish LIMIT 10")


print('\n\n')
print_table_schema(cur, 'MenuJoined', get_table_schema(cur, 'MenuJoined'))
print_query(cur, "SELECT * FROM MenuJoined LIMIT 10")
    
print('\n\n')
# Example join.
res = cur.execute("SELECT * FROM menuPage INNER JOIN menuItem on menuPage.id = menuItem.menu_page_id LIMIT 10")
for i, r in enumerate(res.fetchall()[:10]):
    print(f'{i}: {r}')
    

## List Unclean Schema and Examples

In [ ]:
# Fetch some sample results.
print('\n\n')
print_table_schema(ucur, 'menu_item_uncleaned', get_table_schema(ucur, 'menu_item_uncleaned'))
print_query(ucur, "SELECT * FROM menu_item_uncleaned LIMIT 10")

# Fetch some sample results.
print('\n\n')
print_table_schema(ucur, 'menu_uncleaned', get_table_schema(ucur, 'menu_uncleaned'))
print_query(ucur, "SELECT * FROM menu_uncleaned LIMIT 10")
# print_query(ucur, "SELECT * FROM menu_uncleaned WHERE sponsor = 'ARCHITECTURAL LEAGUE OF NEW YORK' LIMIT 10")
    
# Fetch some sample results.
print('\n\n')
print_table_schema(ucur, 'menu_page_uncleaned', get_table_schema(ucur, 'menu_page_uncleaned'))
print_query(ucur, "SELECT * FROM menu_page_uncleaned LIMIT 10")

# Fetch some sample results.
print('\n\n')
print_table_schema(ucur, 'dish_uncleaned', get_table_schema(ucur, 'dish_uncleaned'))
print_query(ucur, "SELECT * FROM dish_uncleaned LIMIT 10")


print('\n\n')
print_table_schema(ucur, 'MenuJoined', get_table_schema(ucur, 'MenuJoined'))
print_query(ucur, "SELECT * FROM MenuJoined LIMIT 10")

# Use Case Queries

In [ ]:
target_foods = ['eggs', 'coffee', 'steak', 'apple pie']

all_sponsors = [
    'club',  # Generally private social/dinner clubs.
    'hotel',
    'rail',
    'restaurant',
    'ship',
    '',  # blank
]
all_events = [
    'anniversary',  # Generally private event
    'annual event',  # Generally private event
    'banquet',  # Generally private event
    'breakfast',
    'brunch',
    'daily menu',
    'diner',
    'dinner',
    'lunch',
    'private',
    'supper',
    'wine',
    '',  # (blank)
]
all_venues = [
    'airline',
    'club',  # Generally private social/dinner clubs.
    'commercial',
    'edu',  # Generally private event
    'government',  # Generally private event
    'hotel',
    'military',  # Generally private event
    'null',
    'patriotic',  # Generally private event
    'private',
    'professional',  # Generally private event
    'railroad',
    'religious',  # Generally private event
    'restaurant',
    'royal',  # Generally private event
    'ship',
    'social',  # Often a banquet, ball, anniversary, annual meeting, or prof event.
    '',  # (blank)
]
    
public_sponsors = ['hotel', 'rail', 'restaurant', 'ship', '']
public_events = ['breakfast', 'brunch', 'daily menu', 'diner', 'dinner', 'lunch', 'supper', 'wine', '']
public_venues = [ 'airline', 'commercial', 'hotel', 'railroad', 'restaurant', 'ship', '']
quote = lambda x: '\'' + x + '\''
public_sponsors_as_sql_in = ''.join(['(', ','.join([quote(x) for x in public_sponsors]), ')'])
public_events_as_sql_in =  ''.join(['(', ','.join([quote(x) for x in public_events]), ')'])
public_venues_as_sql_in =  ''.join(['(', ','.join([quote(x) for x in public_venues]), ')'])

In [ ]:
u1_query = """
WITH AllVenues AS (
    SELECT *
    FROM
      MenuJoined
    WHERE
      name IS NOT NULL AND
      name LIKE '%{0}%'
    LIMIT 1000
),
AllPublicVenues AS (
    SELECT *
    FROM
        AllVenues
    WHERE
        AllVenues.sponsor IN {1} OR
        AllVenues.event IN {2} OR
        AllVenues.venue IN {3}
),
-- Now we focus on price data. Condense the 
PriceData AS (
    SELECT
        IIF(price IS NOT NULL, price, 0) AS price, 
        IIF(high_price IS NOT NULL, high_price, 0) AS high_price,
        IIF(lowest_price IS NOT NULL, lowest_price, 0) AS lowest_price, 
        IIF(highest_price IS NOT NULL, highest_price, 0) AS highest_price
    FROM
        AllPublicVenues
),
AveragePriceData AS (
    SELECT
        ROUND(AVG(price), 2) AS avg_price,
        ROUND(AVG(high_price), 2) AS avg_high,
        ROUND(AVG(lowest_price), 2) AS avg_lowest,
        ROUND(AVG(highest_price), 2) AS avg_highest
    FROM
        PriceData
),
Results AS (
    SELECT
        *
    FROM
        AllPublicVenues CROSS JOIN
        AveragePriceData
)
SELECT
    name, sponsor, event, venue, avg_price, avg_high, avg_lowest, avg_highest,
    last_appeared, times_appeared
FROM
    Results
ORDER BY
    last_appeared,
    times_appeared
    
DESC
;
"""
u1_query_fmt = u1_query.format(target_foods[0], public_sponsors_as_sql_in, public_events_as_sql_in, public_venues_as_sql_in)

res = cur.execute(u1_query_fmt)
print("name, sponsor, event, venue, avg_price, avg_high, avg_lowest, avg_highest, last_appeared")
for i, r in enumerate(res.fetchall()[:10]):
    print(f'{i}: {r}')

In [ ]:
u1_query_fmt = u1_query.format(target_foods[1], public_sponsors_as_sql_in, public_events_as_sql_in, public_venues_as_sql_in)

res = cur.execute(u1_query_fmt)
print("name, sponsor, event, venue, avg_price, avg_high, avg_lowest, avg_highest, last_appeared")
for i, r in enumerate(res.fetchall()[:10]):
    print(f'{i}: {r}')

In [ ]:
# Uncleaned same query 
# u1_unclean_query_fmt = u1_query.format(target_foods[0], public_sponsors_as_sql_in, public_events_as_sql_in, public_venues_as_sql_in)
# 
# res = ucur.execute(u1_unclean_query_fmt)
# print("name, sponsor, event, venue, avg_price, avg_high, avg_lowest, avg_highest, last_appeared")
# for i, r in enumerate(res.fetchall()[:10]):
#     print(f'{i}: {r}')

## Validation Queries

### Primary Key Constraints



Here we show that the number of rows from all tables that do not appear in the final joined table. This means that their Primary Key (id) columns were not present in other tables as Foreign Keys (e.g. MenuPage.menu_id, MenuItem.menu_page_id, MenuItem.dish_id, Dish.menu_item_id).

In [ ]:
pk_valiation_query_unclean_2 = """
    WITH MenusWithNoPages AS (
        SELECT DISTINCT Menu.id  --, MenuPage.menu_id
        FROM
            menu_uncleaned AS Menu LEFT OUTER JOIN
            menu_page_uncleaned AS MenuPage ON Menu.id = MenuPage.menu_id
        WHERE
            MenuPage.menu_id IS NULL
    )
    SELECT COUNT(*)
    FROM 
        MenusWithNoPages
    ;
"""

pk_valiation_query_unclean_3 = """
    WITH PagesWithNoMenus AS (
        -- SELECT DISTINCT Menu.id, MenuPage.menu_id
        SELECT DISTINCT MenuPage.menu_id
        FROM
            menu_page_uncleaned AS MenuPage LEFT OUTER JOIN 
            menu_uncleaned AS Menu ON MenuPage.menu_id = Menu.id
        WHERE
            Menu.id IS NULL
    )
    SELECT COUNT(*)
    FROM 
        PagesWithNoMenus
    ;
"""

pk_valiation_query_unclean_4 = """
    WITH PagesWithNoItems AS (
        -- SELECT DISTINCT MenuPage.id, MenuItem.id
        SELECT DISTINCT MenuPage.id
        FROM
            menu_page_uncleaned AS MenuPage LEFT OUTER JOIN 
            menu_item_uncleaned AS MenuItem ON MenuPage.id = MenuItem.menu_page_id
        WHERE
            MenuItem.menu_page_id IS NULL
    )
    SELECT COUNT(*)
    FROM 
        PagesWithNoItems
    ;
"""

pk_valiation_query_unclean_5 = """
    WITH ItemsWithNoPages AS (
        -- SELECT DISTINCT MenuPage.id, MenuItem.id
        SELECT DISTINCT MenuItem.id
        FROM
            menu_item_uncleaned AS MenuItem LEFT OUTER JOIN 
            menu_page_uncleaned AS MenuPage ON MenuItem.menu_page_id = MenuPage.id
        WHERE
            MenuPage.id IS NULL
    )
    SELECT COUNT(*)
    FROM 
        ItemsWithNoPages
    ;
"""

pk_valiation_query_unclean_6 = """
    WITH MenusWithNoPages AS (
        SELECT DISTINCT Menu.id, MenuPage.menu_id
        FROM
            menu_uncleaned AS Menu LEFT OUTER JOIN
            menu_page_uncleaned AS MenuPage ON Menu.id = MenuPage.menu_id
        WHERE
            MenuPage.menu_id IS NULL
    ),
    PagesWithNoMenus AS (
        SELECT DISTINCT Menu.id, MenuPage.menu_id
        FROM
            menu_page_uncleaned AS MenuPage LEFT OUTER JOIN 
            menu_uncleaned AS Menu ON MenuPage.menu_id = Menu.id
        WHERE
            Menu.id IS NULL
    ),
    Unified AS (
        SELECT id, menu_id FROM MenusWithNoPages
        UNION ALL
        SELECT id, menu_id FROM PagesWithNoMenus
    )
    SELECT 
        SUM(IIF(id IS NOT NULL, 1, 0)) AS MenusMissingPages,
        SUM(IIF(menu_id IS NOT NULL, 1, 0)) AS PagesMissingMenus
        -- IIF(price IS NOT NULL, price, 0) AS price, 
        -- IIF(high_price IS NOT NULL, high_price, 0) AS high_price,
        -- IIF(lowest_price IS NOT NULL, lowest_price, 0) AS lowest_price, 
        -- IIF(highest_price IS NOT NULL, highest_price, 0) AS highest_price
    FROM 
       Unified 
    ;
"""

pk_valiation_query_unclean_7 = """
    WITH PagesWithNoItems AS (
        SELECT DISTINCT
            MenuPage.id AS page_id, MAX(MenuItem.id) AS item_id
        FROM
            menu_page_uncleaned AS MenuPage LEFT OUTER JOIN 
            menu_item_uncleaned AS MenuItem  ON MenuPage.id = MenuItem.menu_page_id
        WHERE
            MenuItem.menu_page_id IS NULL
        GROUP BY
            MenuPage.id
    ),
    ItemsWithNoPages AS (
        SELECT DISTINCT
            MAX(MenuPage.id) AS page_id, MenuItem.id AS item_id
        FROM
            menu_item_uncleaned AS MenuItem LEFT OUTER JOIN 
            menu_page_uncleaned AS MenuPage ON MenuItem.menu_page_id = MenuPage.id
        WHERE
            MenuPage.id IS NULL
        GROUP BY
            MenuItem.id
    ),
    Unified AS (
        SELECT page_id, item_id FROM PagesWithNoItems
        UNION ALL
        SELECT page_id, item_id FROM ItemsWithNoPages
    )
    SELECT
        SUM(IIF(page_id IS NOT NULL, 1, 0)) AS PagesMissingItems,
        SUM(IIF(item_id IS NOT NULL, 1, 0)) AS ItemsMissingPages
    FROM 
       Unified 
    ;
"""

pk_valiation_query_unclean_8 = """
    WITH ItemsMissingDishes AS (
        SELECT DISTINCT
            MenuItem.dish_id AS item_id, Dish.id AS dish_id
        FROM
            menu_item_uncleaned AS MenuItem LEFT OUTER JOIN 
            dish_uncleaned AS Dish ON MenuItem.dish_id = Dish.id
        WHERE
            Dish.id IS NULL
    ),
    DishesMissingItems AS (
        SELECT DISTINCT
            MenuItem.dish_id AS item_id, Dish.id AS dish_id
        FROM
             dish_uncleaned AS Dish LEFT OUTER JOIN 
             menu_item_uncleaned AS MenuItem ON Dish.id = MenuItem.dish_id
        WHERE
            MenuItem.dish_id IS NULL
    ),
    Unified AS (
        SELECT item_id, dish_id FROM ItemsMissingDishes
        UNION ALL
        SELECT item_id, dish_id FROM DishesMissingItems 
    )
    SELECT
        SUM(IIF(item_id IS NOT NULL, 1, 0)) AS ItemsMissingDishes,
        SUM(IIF(dish_id IS NOT NULL, 1, 0)) AS DishesMissingItems 
    FROM 
       Unified 
    ;
"""


# # Result on cleaned dataset.
# pk_valiation_query_clean_fmt_1 = pk_valiation_query_clean_1.format()
# print_query(cur, pk_valiation_query_clean_fmt_1)
    
# Menus with no corresponding MenuPages.
print('MenusWithNoPages.')
pk_valiation_query_unclean_fmt_2 = pk_valiation_query_unclean_2.format()
print_query(ucur, pk_valiation_query_unclean_fmt_2, limit=200)

# MenuPages with no corresponding Menu.
print('PagesWithNoMenus')
pk_valiation_query_unclean_fmt_3 = pk_valiation_query_unclean_3.format()
print_query(ucur, pk_valiation_query_unclean_fmt_3, limit=200)

# MenuPages with no corresponding MenuItems. 
print('PagesWithNoItems')
pk_valiation_query_unclean_fmt_4 = pk_valiation_query_unclean_4.format()
print_query(ucur, pk_valiation_query_unclean_fmt_4, limit=200)

# MenuItems with no corresponding MenuPage. 
print('ItemsWithNoPages')
pk_valiation_query_unclean_fmt_5 = pk_valiation_query_unclean_5.format()
print_query(ucur, pk_valiation_query_unclean_fmt_5, limit=200)

print('\n\n')

# Menus with no corresponding MenuPages.
# MenuItems with no corresponding MenuPage. 
print('Denial Constraints - MenusMissingPages and PagesMissignMenus')
pk_valiation_query_unclean_fmt_6 = pk_valiation_query_unclean_6.format()
print_query(ucur, pk_valiation_query_unclean_fmt_6, limit=200)

# MenuPages with no corresponding MenuItems. 
# MenuItems with no corresponding MenuPage. 
print('Denial Constraints - PagesMissingItems and ItemsMissingPages')
pk_valiation_query_unclean_fmt_7 = pk_valiation_query_unclean_7.format()
print_query(ucur, pk_valiation_query_unclean_fmt_7, limit=200)

# MenuItems with no corresponding Dishes. 
# Dishes with no corresponding MenuItems. 
print('Denial Constraints - MenuItemsMissingDishes and DishesMissingMenuItems')
pk_valiation_query_unclean_fmt_8 = pk_valiation_query_unclean_8.format()
print_query(ucur, pk_valiation_query_unclean_fmt_8, limit=200)


We can be sure that our cleaned queries do not contain such primary/foreign key violations because of the way we define our temporary table `MenuJoined`.

We write all our use case queries against the temporary table `MenuJoined` which is defined as follows.

```sql
DROP TABLE IF EXISTS MenuJoined;
CREATE TEMP TABLE MenuJoined AS
SELECT
    Menu.id AS MenuId,
    menuPage.id AS MenuPageId,
    menuItem.id AS MenuItemId,
    Dish.id AS DishId,
    *
FROM
  nypl_menu_step_2_sql_export_demo AS Menu INNER JOIN
  menuPage ON Menu.id = menuPage.id INNER JOIN
  menuItem ON menuPage.id = menuItem.menu_page_id INNER JOIN
  Dish ON Dish.id = menuItem.dish_id
;
```

By definition we are using INNER JOINs across all tables. This means that any MenuPage not associated with a valid `Menu.id` will not be in the result set. Similarly the dropped `MenuPage` implies that some `MenuItems` may not have a valid `menu_page_id` field (as the matching `MenuPage.id` row no longer exists in the MenuPage table). And so on for `MenuItem.dish_id`.

Any constrain violations therefore do not need to filtered out of the data using Denial Constraints before hand. We can simply choose not to `JOIN` the invalid rows. We believe this is a less error prone way of cleaning the data, by creating a temporary table with only the constraint-obeying rows we intend to query.

### Unique ID Constraint

In [ ]:
# Validation Queries
# Check whether menuPage Id is unique or not

def is_menuPage_id_unique():
    # Connect to the database
    conn = sqlite3.connect(menu_db_path)
    cursor = conn.cursor()

    # Execute the SELECT query to count occurrences of each "id" value in the "menuPage" table
    count_query = """
    SELECT COUNT(id) FROM (
        SELECT id FROM menuPage GROUP BY id HAVING COUNT(id) > 1
    );
    """
    cursor.execute(count_query)

    # Fetch the result (the count of duplicate "id" values)
    count_of_duplicates = cursor.fetchone()[0]

    # Close the connection
    conn.close()

    # If there are no duplicates (count_of_duplicates == 0), then the "id" values are unique
    is_unique = count_of_duplicates == 0

    return is_unique


# Check whether menuItem Id is unique or not
def is_menuItem_id_unique():
    # Connect to the database
    conn = sqlite3.connect(menu_db_path)
    cursor = conn.cursor()

    # Execute the SELECT query to count occurrences of each "id" value in the "menuItem" table
    count_query = """
    SELECT id, COUNT(id) AS occurrences
    FROM menuItem
    GROUP BY id;
    """
    cursor.execute(count_query)

    # Fetch all the rows
    rows = cursor.fetchall()

    # Check if all occurrences are equal to 1
    is_unique = all(occurrences == 1 for id, occurrences in rows)

    # Close the connection
    conn.close()

    return is_unique


# Check whether dish Id is unique or not
def is_dish_id_unique():
    # Connect to the database
    conn = sqlite3.connect(menu_db_path)
    cursor = conn.cursor()

    # Execute the SELECT query to find any duplicate "id" values in the "dish" table
    duplicate_query = """
    SELECT id FROM dish
    GROUP BY id
    HAVING COUNT(id) > 1;
    """

    cursor.execute(duplicate_query)

    # Fetch any duplicate "id" values
    duplicate_ids = cursor.fetchall()

    # Close the connection
    conn.close()

    # If there are no duplicate "id" values, then the "id" values are unique
    is_unique = len(duplicate_ids) == 0

    return is_unique


# Check whether dish Id is unique or not
def is_menu_id_unique():
    # Connect to the database
    conn = sqlite3.connect(menu_db_path)
    cursor = conn.cursor()

    # Execute the SELECT query to find any duplicate "id" values in the "nypl_menu_step_2_sql_export_demo" table
    duplicate_query = """
    SELECT id FROM nypl_menu_step_2_sql_export_demo
    GROUP BY id
    HAVING COUNT(id) > 1;
    """
    cursor.execute(duplicate_query)

    # Fetch any duplicate "id" values
    duplicate_ids = cursor.fetchall()

    # Close the connection
    conn.close()

    # If there are no duplicate "id" values, then the "id" values are unique
    is_unique = len(duplicate_ids) == 0

    return is_unique

In [ ]:
# Call the function to check if menuPage.id is unique
result = is_menuPage_id_unique()
print("Is menuPage id unique?", result)

# Call the function to check if menuItem.id is unique
result = is_menuItem_id_unique()
print("Is menuItem id unique?", result)

# Call the function to check if dish.id is unique
result = is_dish_id_unique()
print("Is dish id unique?", result)

# Call the function to check if nypl_menu_step_2_sql_export_demo.id is unique
result = is_menu_id_unique()
print("Is nypl_menu_step_2_sql_export_demo.id unique?", result)


### Menu


Here we show that the number of rows returned for a query with
  1. a sponsor type containing the word "club"
  2. a event or sponsor field containing the regex `[Dd]inn` for `dinner`
  
is lower for cleaned data than uncleaned data.
In the Menu dataset a sponsor type containing the word "club" is almost
always a private event. E.g. a dinner for a local badminton club. In the cleaned
dataset we filter over 50% of the misleading club results.
The remaining clubs in the cleaned data appear to refer to nightclubs or dinner clubs
that are more frequently open to the public.

In [ ]:
menu_valiation_query_1 = """
    WITH ClubPublicVenues AS (
        SELECT *
        FROM
            MenuJoined
        WHERE
            sponsor LIKE '%club%' AND 
            (sponsor LIKE '%dinn%' OR
             sponsor LIKE '%Dinn%' OR
             event LIKE '%dinn%' OR
             event LIKE '%Dinn%')
    )
    SELECT COUNT(*)
    FROM 
        ClubPublicVenues
"""

# Result on cleaned dataset.
menu_valiation_query_fmt_1 = menu_valiation_query_1.format()
print_query(cur, menu_valiation_query_fmt_1)
    
# Result on uncleaned dataset.
menu_valiation_query_fmt_1 = menu_valiation_query_1.format()
print_query(ucur, menu_valiation_query_fmt_1)

Here we show that the number of rows with sponsor + event + venue
all NULL has increased in the cleaned data. This a a result of clutering
by the sponsor + event + event fields into a smaller subset of types/enums/categories
the clustering and then thresholding/removal of the long tail of types that
are referenced by a handful of row (e.g. < 10) makes the place data more accurate.

More accurate place data helps our use case U1 by only returning relevant public
venues when the user enters a dish-name query.

In [ ]:
menu_validation_query_2 = """
    WITH NullPlaceInformation AS (
        SELECT *
        FROM
            MenuJoined
        WHERE
            (sponsor = '' OR sponsor IS NULL) AND
            (event = '' OR event IS NULL) AND
            (venue = '' OR venue IS NULL)
    )
    SELECT COUNT(*)
    FROM 
        NullPlaceInformation
"""
# Result on cleaned dataset.
menu_valiation_query_fmt_2 = menu_valiation_query_2.format()
print_query(cur, menu_valiation_query_fmt_2)
    
# Result on uncleaned dataset.
menu_valiation_query_fmt_2 = menu_valiation_query_2.format()
print_query(ucur, menu_valiation_query_fmt_2)

Here we show that a set of handpicked private events that all contain
the regex `[Dd]inner` in the `event` column do not appear in the cleaned
dataset because they do not refer to private events.
The cleaned data correctly clusters these rows as probably not private.

Looking at the results below we can see that both queries return the specified
rows because in both cases the event field contains `DINNER, Dinner, or dinner`.

Comparing the sponsor, and venues field though

```
public_sponsor = ['hotel', 'rail', 'restaurant', 'ship', '']
public_venues = [ 'airline', 'commercial', 'hotel', 'railroad', 'restaurant', 'ship', '']
```

The cleaned rows have `sponsor` of type military, political, and social, which are not
included in the list of public sponsor types used in U1.

In [ ]:
# Menu Validation

# Here we show that a set of handpicked private events that all contain
# the regex "[Dd]inner" in the `event` column do not appear in the cleaned
# dataset because they do not refer to private events.
# The cleaned data correctly clusters these rows as probably not private.
menu_valiation_query_3 = """
    WITH PrivateDinnerEvents AS (
        SELECT *
        FROM
            MenuJoined
        WHERE
            --(event LIKE '%dinne%' OR event LIKE '%Dinne%' OR event LIKE '%DINNER%') AND
            MenuId IN (22770, 25288, 21689, 21744)
    )
    SELECT *
    FROM 
         PrivateDinnerEvents
    GROUP BY
        MenuId
"""
# Result on cleaned dataset.
print("Cleaned")
menu_valiation_query_fmt_3 = menu_valiation_query_3.format()
print_query(cur, menu_valiation_query_fmt_3, limit=10)
print('\n\n')
    
# Result on uncleaned dataset.
print("Uncleaned")
menu_valiation_query_fmt_3 = menu_valiation_query_3.format()
print_query(ucur, menu_valiation_query_fmt_3, limit=10)

TBD uppercase vs. lowercase, underscores, punctuation differences in search. dinner vs DINNER.

In [ ]:
# Menu Validation



### MenuItem

In [ ]:
# Check if price on menuItem page is greater than 0
def check_menuItem_price_greater_than_zero():
    # Connect to the database
    conn = sqlite3.connect(menu_db_path)
    cursor = conn.cursor()

    # Execute the SELECT query to check if there are rows with "price" greater than zero
    query = """
    SELECT EXISTS (SELECT 1 FROM menuItem WHERE price > 0) AS result;
    """
    cursor.execute(query)

    # Fetch the result
    result = cursor.fetchone()[0] == 1

    # Close the connection
    conn.close()

    return result


# Call the function to check if there are rows with "price" greater than zero
result = check_menuItem_price_greater_than_zero()
print("Are all the prices in menuItem table greater than zero?:", result)

### Dish

Here we check that the name column in cleaned Dish table returns lower number of dishes 
with chicken compared to original Dish table. 

In [ ]:
dish_valiation_query_1 = """
    WITH DishCount AS (
        SELECT *
        FROM
            MenuJoined
        WHERE
            name LIKE '%chicken%' OR
            name LIKE '%Chicken%' OR
            name LIKE '%CHICKEN%'
            
    )
    SELECT COUNT(*)
    FROM 
        DishCount
"""

# Result on cleaned dataset.
dish_valiation_query_fmt_1 = dish_valiation_query_1.format()
print_query(cur, dish_valiation_query_fmt_1)
    
# Result on uncleaned dataset.
dish_valiation_query_fmt_1 = dish_valiation_query_1.format()
print_query(ucur, dish_valiation_query_fmt_1)

Here we check if the number of times a dish appeared is less than the number of menus the 
dish appeared on. "times_appeared" should always be equal or greater than number of menus
it appeared on. Checking "times_appeared < menus_appeared" gives us 0 for cleaned data
but for the uncleaned data we get some items.

In [ ]:
dish_valiation_query_2 = """
    WITH DishAppearance AS (
        SELECT *
        FROM
            MenuJoined
        WHERE
            times_appeared < menus_appeared
    )
    SELECT COUNT(*)
    FROM 
        DishAppearance
"""

# Result on cleaned dataset.
dish_valiation_query_fmt_2 = dish_valiation_query_2.format()
print_query(cur, dish_valiation_query_fmt_2)
    
# Result on uncleaned dataset.
dish_valiation_query_fmt_2 = dish_valiation_query_2.format()
print_query(ucur, dish_valiation_query_fmt_2)

Dish last_appeared < first_appeared
Here we compare the first and last time a dish appeared on a menu. 

In [ ]:
dish_valiation_query_3 = """
    WITH FirstLastAppearance AS (
        SELECT *
        FROM
            MenuJoined
        WHERE
            last_appeared < first_appeared 
    )
    SELECT COUNT(*)
    FROM 
        FirstLastAppearance
"""

# Result on cleaned dataset.
dish_valiation_query_fmt_3 = dish_valiation_query_3.format()
print_query(cur, dish_valiation_query_fmt_3)
    
# Result on uncleaned dataset.
dish_valiation_query_fmt_3 = dish_valiation_query_3.format()
print_query(ucur, dish_valiation_query_fmt_3)

# Load Data

In [ ]:
_DISH_CSV_FILE = 'dish_uncleaned.csv'
_MENU_CSV_FILE = 'menu_uncleaned.csv'
_MENU_ITEM_CSV_FILE = 'menu_item_uncleaned.csv'
_MENU_PAGE_CSV_FILE = 'menu_page_uncleaned.csv'
path = os.path.join(os.path.dirname(os.getcwd()), 'data', _DISH_CSV_FILE)
dish_df = pd.read_csv(path)
path = os.path.join(os.path.dirname(os.getcwd()), 'data', _MENU_CSV_FILE)
menu_df = pd.read_csv(path)
path = os.path.join(os.path.dirname(os.getcwd()), 'data', _MENU_ITEM_CSV_FILE)
mi_df = pd.read_csv(path)
path = os.path.join(os.path.dirname(os.getcwd()), 'data', _MENU_PAGE_CSV_FILE)
mp_df = pd.read_csv(path)

In [ ]:
dish_df

In [ ]:
menu_df

In [ ]:
mi_df

In [ ]:
#mp_df['page_number'] = mp_df['page_number'].astype('int')
print(f"null pages: {mp_df['page_number'].isnull().sum()}")
mp_df

# Joining

In [ ]:
#all_items = pd.concat([mp_df.set_index('id'), mi_df.set_index('menu_page_id')], axis=1, join='inner')
all_pages = mp_df.merge(mi_df, left_on='id', right_on='menu_page_id', how='inner').reset_index(drop=True)
all_pages = all_pages.drop(labels=['id_x', 'id_y'], axis='columns')
# all_pages

In [ ]:
all_menus = all_pages.merge(menu_df, left_on='menu_id', right_on='id', how='inner').reset_index(drop=True)
all_menus = all_menus.drop(labels=['id'], axis='columns')
# all_menus

In [ ]:
all_items = all_menus.merge(dish_df, left_on='dish_id', right_on='id', how='inner')
all_items = all_items.drop(labels=['id'], axis='columns')
# print(all_items.columns)
# all_items

# Filtering

In [ ]:
# The following sections enable individual filtering steps.
FILTER_RM_TITLES_WITHOUT_YEARS = True
MUTATION_ADD_YEAR_COL = True

# All filters/mutations will be applied to the fdf (filtered data frame). The original will be preserved.
fdf = df.copy()

In [ ]:
titles = fdf['title']
titles[:100]

In [ ]:
# Find titles with the year in them
total_titles = fdf['title'].size
null_titles = fdf.title.isnull().sum()
title_with_year = fdf['title'].str.contains('[012]\d{2,3}')
num_titles_with_year = title_with_year.sum()
pct_with_year = num_titles_with_year / (total_titles - null_titles)
print(f'total: {total_titles}, nulls: {null_titles},  '
      f'num_with_year: {num_titles_with_year}, % w/yr: {pct_with_year:0.4}%')

In [ ]:
if FILTER_RM_TITLES_WITHOUT_YEARS:
    fdf = fdf[fdf['title'].notna()]
    title_with_year = fdf['title'].str.contains('[012]\d{2,3}')
    fdf = fdf[title_with_year]
    
if MUTATION_ADD_YEAR_COL:
    if FILTER_RM_TITLES_WITHOUT_YEARS:
        fdf['year'] = fdf.title.str.extract(pat='([012]\d{2,3})', expand=False)
        #fdf['year'] = pd.to_datetime(fdf['year'], format='%y', errors='raise')
        fdf['year'] = fdf['year'].astype('int32')